<h2>Импорт необходимых библиотек</h2>

In [1]:
import geopandas  # библиотека для работы с геопространственными данными
import ipyleaflet  # библиотека интерактивных виджетов для карты, основанная на ipywidgets
import ipywidgets  # библиотека интерактивных HTML-виджетов для ноутбуков Jupyter и ядра IPython
import os  # библиотека функций для работы с операционной системой
import psycopg2  # библиотека для связи с базой данных

from contextlib import closing  # функция для закрытия соединения с БД при выходе из менеджера контекста
from dotenv import load_dotenv  # считывания пар ключ-значение из файла .env и установки их как переменные среды
from pyproj import Geod  # выполнение прямых и обратных геодезических вычислених (длина фигурной геометрии)
from typing import Callable, Optional, Union  # для тайп-хинтингов Python

### Игнорируем предупреждения


In [2]:
import warnings
warnings.filterwarnings('ignore')

<h2>Загрузка данных по зданиям города Познань</h2>

### Вариант 1. Загружаем напрямую из файла

In [3]:
# df_buildings = geopandas.read_file(filename='./shapefiles/buildings.shp')
# df_buildings.insert(0, 'id_building', range(1, len(df_buildings) + 1))

# df_buildings = df_buildings.sample(20000) # выборка случайных 20к записей

# df_buildings.info() # общая информация по датафрейму
# df_buildings.head() # первые пять записей датафрейма

### Вариант 2. Загружаем из базы данных

In [4]:
# Получаем содержимое указанной таблицы из БД
def get_table_data_from_db(table_name: str = "buildings") -> Optional[geopandas.GeoDataFrame]:
    load_dotenv()
    with closing(psycopg2.connect(dbname=os.environ.get('DB_NAME'), 
                                  user=os.environ.get('DB_USER'), 
                                  password=os.environ.get('DB_PASSWORD'))) as conn:
        try:
            sql_query = f"SELECT * FROM {table_name};"
            df_table = geopandas.read_postgis(sql=sql_query, con=conn, geom_col='geom')
            return df_table
        except Exception as e:
            print(e)
            
if (df_buildings := get_table_data_from_db()) is None:
    raise ValueError("Не удалось извлечь данные из указанной таблицы. Проверьте правильность запроса.")
    
df_buildings = df_buildings.sample(20000) # выборка случайных 20к записей

df_buildings.info() # общая информация по датафрейму
df_buildings.head() # первые пять записей датафрейма

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 20000 entries, 77566 to 140573
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   gid     20000 non-null  int64   
 1   geom    20000 non-null  geometry
 2   osm_id  20000 non-null  int64   
 3   name    248 non-null    object  
 4   type    4991 non-null   object  
dtypes: geometry(1), int64(2), object(2)
memory usage: 937.5+ KB


,gid,geom,osm_id,name,type
77566,224682,"MULTIPOLYGON (((16.87596 52.45313, 16.87599 52...",1086198943,None,outbuilding
70384,217500,"MULTIPOLYGON (((17.12324 52.35903, 17.12325 52...",1066984129,None,outbuilding
53848,200964,"MULTIPOLYGON (((16.82037 52.41213, 16.82041 52...",941127919,None,None
82403,2277,"MULTIPOLYGON (((16.86099 52.41689, 16.86101 52...",37517107,None,None
225459,145340,"MULTIPOLYGON (((17.08150 52.40326, 17.08150 52...",669282579,None,residential


<h2>Работа над картой</h2>

In [5]:
main_map = ipyleaflet.Map(basemap=ipyleaflet.basemaps.OpenStreetMap.Mapnik,
                          center=(52.4126, 16.9219),
                          zoom=10,
                          max_zoom=18, 
                          scroll_wheel_zoom=True)

# Добавляем панель контроля шаров для возможности скрытия последних
layers_control = ipyleaflet.LayersControl(position='topleft')
main_map.add_control(layers_control)
# Добавляем возможность открытия карты на весь экран
main_map.add_control(ipyleaflet.FullScreenControl())

<h3>Добавление слоя с данными по зданиям города Познань</h3>

In [6]:
# Создаём и добавляем шар с отмеченными зданиями города Познань
geo_data = ipyleaflet.GeoData(geo_dataframe=df_buildings,
                              style={'color': 'red', 'fillColor': '#3366cc', 'opacity': 0.5,
                                     'weight': 1.9, 'dashArray': '2', 'fillOpacity': 0.6
                              },
                              hover_style={
                                  'fillColor': 'orange' , 'fillOpacity': 0.2
                              },
                              name='Buildings')
main_map.add_layer(geo_data)

<h3>Слой маркеров: добавление/удаление, кластер маркеров, функции-слушатели</h3>

In [7]:
# Создание кластера маркеров, который будет содержать все маркеры, созданные пользователем
user_markers_cluster = ipyleaflet.MarkerCluster(name='User Markers')
main_map.add_layer(user_markers_cluster)

# Создание шаблона поп-апа
popup_text_template = ipywidgets.HTML()


# Функция добавление маркера в кластер маркеров пользователя
def add_marker_to_cluster(marker: ipyleaflet.leaflet.Marker) -> None:
    current_markers_in_cluster = list(user_markers_cluster.markers)
    current_markers_in_cluster.append(marker)
    user_markers_cluster.markers = current_markers_in_cluster

    
# Функция удаления маркера из кластера маркеров пользователя
def remove_marker_from_cluster(marker: ipyleaflet.leaflet.Marker) -> None:
    current_markers_in_cluster = list(user_markers_cluster.markers)
    current_markers_in_cluster.remove(marker)
    user_markers_cluster.markers = current_markers_in_cluster
    
    
# Функция подстановки текста в поп-ап маркера
def set_popup_text(element: ipyleaflet.leaflet.Marker, text: str) -> None:
    popup_text_template.value = text
    element.popup = popup_text_template
    
    
# Функция реакции на перемещение маркера, созданного пользователем
def on_marker_move(marker: ipyleaflet.leaflet.Marker) -> None:
    def callback(*args, **kwargs) -> Callable:
        # Установка текста поп-апа для маркера
        set_popup_text(marker, str(kwargs.get("location")))
    return callback

# Функция реакции на двойной клик по маркеру
def on_marker_double_click(marker: ipyleaflet.leaflet.Marker) -> None:
    def callback(*args, **kwargs) -> Callable:
        # Удаление маркера из кластера
        remove_marker_from_cluster(marker)
    return callback


# Функция реакции нажатия на карту -> добавление маркера в точке клика
def on_map_click(**kwargs):
    if kwargs.get('type') == 'click':
        # Создаём маркер в точке клика
        default_marker = ipyleaflet.Marker(location=kwargs.get('coordinates'))
        # Добавляем поп-ап с информацией о координатах
        set_popup_text(default_marker, str(kwargs.get('coordinates')))
        # Устанавливаем функцию-прослушку на перемещение маркера
        default_marker.on_move(on_marker_move(default_marker))
        # Устанавливаем функцию-прослушку на перемещение маркера
        default_marker.on_dblclick(on_marker_double_click(default_marker))
        # Добавляем маркер в соответствующий кластер
        add_marker_to_cluster(default_marker)


# Устанавливаем функцию-прослушку для карты на добавление маркера в точке клика
main_map.on_interaction(on_map_click)

<h3>Создаём и добавляем кнопки управления и поле вывода</h3>

In [8]:
# Создаём кнопки для построения и удаления маршрута
remove_route_button = ipywidgets.widgets.Button(description='Remove Route!',
                                                disabled=False,
                                                button_style='danger')
button_remove_wc = ipyleaflet.WidgetControl(widget=remove_route_button, position='bottomright')
build_route_button = ipywidgets.widgets.Button(description='Build Route!',
                                               disabled=False,
                                               button_style='success')
button_build_wc = ipyleaflet.WidgetControl(widget=build_route_button, position='bottomright')

# Создаём текстовое поле для вывода длины растояния
route_distance_field = ipywidgets.widgets.Text(placeholder='Route Distance', disabled=True)
field_wc = ipyleaflet.WidgetControl(widget=route_distance_field, position='bottomleft')

main_map.add_control(button_remove_wc)
main_map.add_control(button_build_wc)
main_map.add_control(field_wc)

<h3>Слой заданного маршрута: генерация запроса, вызов функции БД, вывод на карте</h3>

In [9]:
# Создаём переменную под шар для ломанной маршрута
geo_route = None


# Функция генерации парметра функции, который содержит координаты всех маркеров
def get_generated_query() -> str:
    query = 'LINESTRING('
    for marker in user_markers_cluster.markers:
        for coordinate in marker.location[::-1]:
            query += f'{coordinate} '
        query += ','
    query = query.rstrip(' ,')
    query += ')'
    return query

# Обращение к функции БД по постройке маршрута
def get_route_form_db() -> Optional[geopandas.GeoDataFrame]:
    load_dotenv()
    with closing(psycopg2.connect(dbname='poznan_gis', 
                                  user=os.environ.get('DB_USER'), 
                                  password=os.environ.get('DB_PASSWORD'))) as conn:
        try:
            sql_query = f"SELECT * FROM build_route_geometry('{get_generated_query()}');"
            route_df = geopandas.GeoDataFrame.from_postgis(sql=sql_query, con=conn, geom_col='build_route_geometry')
            return route_df
        except Exception as e:
            print(e)
        
# Отображение построенного маршрута на карту
def set_route() -> None:
    route_df = get_route_form_db()
    if route_df is not None:
        global geo_route
        if geo_route is None:
            geo_route = ipyleaflet.GeoData(geo_dataframe=route_df, 
                                           style={'color': 'green'},
                                           hover_style={'сolor': 'red' , 'fillOpacity': 0.2},
                                           name='Routes')
            main_map.add_layer(geo_route)
        else:
            geo_route.data.clear()
            geo_route.geo_dataframe = route_df
        route_distance_field.value = f"{Geod(ellps='WGS84').geometry_length(route_df.iloc[0, 0]):.2f} meters"
    

# Функция реакции на клик по кнопке 'Build Route!'
def on_build_route_button_clicked(b) -> None:
    if user_markers_cluster.markers:
        set_route()

        
# Функция реакции на клик по кнопке 'Remove Route!'
def on_remove_route_button_clicked(b) -> None:
    global geo_route
    if geo_route is not None:
        main_map.remove_layer(geo_route)
        geo_route = None
        route_distance_field.value = ''

build_route_button.on_click(on_build_route_button_clicked)
remove_route_button.on_click(on_remove_route_button_clicked)

In [10]:
main_map

Map(center=[52.4126, 16.9219], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…